In [43]:
from rdkit import Chem
import rdkit
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import torch
from torch_geometric.nn import GCNConv,global_max_pool,AGNNConv,NNConv
from torch.nn import ModuleList ,BatchNorm1d,Linear
import torch.nn.functional  as F
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [44]:
# 現在の作業ディレクトリを取得
current_directory = os.getcwd()
# データセットのパスを取得
data_path = os.path.join(current_directory, 'datasets', 'dataset.csv')
#データセットを取得
df = pd.read_csv(data_path)

In [47]:
def smiles_to_atom_feature(smiles : str, res_type='tensor'):
    mol = rdkit.Chem.MolFromSmiles(smiles)
    if mol is None:
        mol = rdkit.Chem.MolFromSmiles('')
    xs = []
    for atom in mol.GetAtoms():
        x = []
        x.append(atom.GetSymbol() == 'B')
        x.append(atom.GetSymbol() == 'C')
        x.append(atom.GetSymbol() == 'F')
        x.append(atom.GetSymbol() == 'H')
        x.append(atom.GetSymbol() == 'N')
        x.append(atom.GetSymbol() == 'O')
        x.append(atom.GetSymbol() == 'P')
        x.append(atom.GetSymbol() == 'S')

        x.append(str(atom.GetChiralTag()) == 'CHI_UNSPECIFIED')
        x.append(str(atom.GetChiralTag()) == 'CHI_TETRAHEDRAL_CW')
        x.append(str(atom.GetChiralTag()) == 'CHI_TETRAHEDRAL_CCW')
        x.append(str(atom.GetChiralTag()) == 'CHI_OTHER')

        for degree in range(11):
            x.append(atom.GetTotalDegree() == degree)

        for formal_charge in range(-5,6):
            x.append(atom.GetFormalCharge() == formal_charge)

        for num_hs in range(7):
            x.append(atom.GetTotalNumHs() == num_hs)

        for num_radical_electrons in range(10):
            x.append(atom.GetNumRadicalElectrons() == num_radical_electrons)
            
        x.append(str(atom.GetHybridization()) == 'UNSPECIFIED')
        x.append(str(atom.GetHybridization()) == 'S')
        x.append(str(atom.GetHybridization()) == 'SP')
        x.append(str(atom.GetHybridization()) == 'SP2')
        x.append(str(atom.GetHybridization()) == 'SP3')
        x.append(str(atom.GetHybridization()) == 'SP3D')
        x.append(str(atom.GetHybridization()) == 'SP3D2')
        x.append(str(atom.GetHybridization()) == 'OTHER')
        
        x.append(atom.GetIsAromatic() == True)

        x.append(atom.IsInRing() == True)

        xs.append(x)
    if res_type == 'tensor':
        xs = torch.tensor(xs,dtype=torch.float32)
    elif res_type == 'list':
        pass

    return xs

In [48]:
def smiles_to_bond_feature(smiles : str):
    edge_index = [[],[]]
    edge_attr = []
    mol = rdkit.Chem.MolFromSmiles(smiles)
    for bond in mol.GetBonds():
        edge_features = []
        edge_index[0].append(bond.GetBeginAtomIdx())
        edge_index[0].append(bond.GetEndAtomIdx())
        edge_index[1].append(bond.GetEndAtomIdx())
        edge_index[1].append(bond.GetBeginAtomIdx())

        edge_features.append(str(bond.GetBondType()) == 'misc')
        edge_features.append(str(bond.GetBondType()) == 'SINGLE')
        edge_features.append(str(bond.GetBondType()) == 'DOUBLE')
        edge_features.append(str(bond.GetBondType()) == 'TRIPLE')
        edge_features.append(str(bond.GetBondType()) == 'AROMATIC')

        
        edge_features.append(str(bond.GetStereo()) == 'STEREONONE')
        edge_features.append(str(bond.GetStereo()) == 'STEREOZ')
        edge_features.append(str(bond.GetStereo()) == 'STEREOE')
        edge_features.append(str(bond.GetStereo()) == 'STEREOCIS')
        edge_features.append(str(bond.GetStereo()) == 'STEREOTRANS')
        edge_features.append(str(bond.GetStereo()) == 'STEREOANY')

        edge_features.append(str(bond.GetIsConjugated()) == True)

        edge_attr.append(edge_features)
        edge_attr.append(edge_features)

    edge_index = torch.tensor(edge_index,dtype=torch.long)
    edge_attr = torch.tensor(edge_attr,dtype=torch.float32)

    return edge_index, edge_attr

In [49]:
class MolecularGCN2(torch.nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim1,hidden_dim2,aggr,num_conv_layers):
        super(MolecularGCN2, self).__init__()
        self.activation_func = F.relu

        edge_fc1 = nn.Linear(edge_feature_dim, node_feature_dim*hidden_dim1)
        # jittable()でscriptに変換できるように
        nnconv1 = NNConv(node_feature_dim, hidden_dim1, edge_fc1, aggr=aggr).jittable()

        self.conv_list = nn.ModuleList()
        self.conv_list.append(nnconv1)

        for _ in range(num_conv_layers - 1):
            edge_fc = nn.Linear(edge_feature_dim, hidden_dim1*hidden_dim1)
            nnconv = NNConv(hidden_dim1, hidden_dim1, edge_fc, aggr=aggr).jittable()
            self.conv_list.append(nnconv)
        
        self.fc1 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc2 = nn.Linear(hidden_dim2, 1)

    def forward(self,  x, edge_index, edge_attr, batch):
        for f in self.conv_list:
            x = f(x=x,edge_index=edge_index, edge_attr=edge_attr)
            x = self.activation_func(x)
        x = global_max_pool(x,batch)
        x = self.fc1(x)
        x = self.activation_func(x)
        x = self.fc2(x)
        return x

model = MolecularGCN2(61,12,num_conv_layers=5, hidden_dim1=70, hidden_dim2= 162, aggr= 'add')

In [50]:
smiles_column = df["SMILES"]

# ヘッダーを含むSMILESデータを作成
smiles_with_header = 'SMILES\n' + '\n'.join(smiles_column)

# 分子を作成
smiles_supplier = Chem.SmilesMolSupplierFromText(smiles_with_header, delimiter=',', titleLine=True, smilesColumn=0, nameColumn=-1)

# 分子データの取得
molecules = [mol for mol in tqdm(smiles_supplier)]

100%|██████████| 198655/198655 [00:11<00:00, 17385.79it/s]


In [78]:
# Convert SMILES strings to atom features and bond features
atom_features = [smiles_to_atom_feature(smiles) for smiles in smiles_column]
bond_features = [smiles_to_bond_feature(smiles) for smiles in smiles_column]

# Separate edge_index and edge_attr into different lists
edge_indices, edge_attrs = zip(*bond_features)

# Find the maximum number of atoms in the molecules
max_atoms_0 = max(feature.size(0) for feature in atom_features)
max_atoms_1 = max(feature.size(1) for feature in atom_features)
max_edges_0 = max(index.size(0) for index in edge_indices)
max_edges_1 = max(index.size(1) for index in edge_indices)
max_attrs_0 = max(attr.size(0) for attr in edge_attrs)
max_attrs_1 = max(attr.size(1) for attr in edge_attrs)

# find max in dim0 and dim1
max_size_0 = max(max_atoms_0, max_edges_0, max_attrs_0)
max_size_1 = max(max_atoms_1, max_edges_1, max_attrs_1)

# Pad the tensors to have the same size
atom_features_padded = [torch.cat([feature, torch.zeros(max_size_0 - feature.size(0), max_size_1 - feature.size(1))]) for feature in atom_features]
edge_indices_padded = [torch.cat([index, torch.zeros(max_size_0 - index.size(0), max_size_1 - index.size(1))]) for index in edge_indices]
edge_attrs_padded = [torch.cat([attr, torch.zeros(max_size_0 - attr.size(0), max_size_1 - attr.size(1))]) for attr in edge_attrs]

# Convert the padded atom features to a PyTorch tensor
atom_features_tensor = torch.stack(atom_features_padded)

bond_features_tensor = torch.cat([edge_indices_padded])

# Convert the list to a PyTorch tensor
bond_features_tensor = torch.stack(bond_features_tensor)

In [57]:
# Split the data into training and testing sets
train_indices, test_indices = train_test_split(range(len(molecules)), test_size=0.2, random_state=42)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(atom_features_tensor[train_indices], bond_features_tensor[train_indices])
test_dataset = TensorDataset(atom_features_tensor[test_indices], bond_features_tensor[test_indices])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# モデルの構築
model = MolecularGCN2(61, 12, num_conv_layers=5, hidden_dim1=70, hidden_dim2=162, aggr='add')

# 損失関数の定義
criterion = nn.MSELoss()

# 最適化アルゴリズムの選択
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習のループ
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    for data in train_loader:
        inputs, targets = data
        optimizer.zero_grad()
        outputs = model(inputs)  # Adjust as needed
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # エポックごとにモデルの評価
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for data in test_loader:
            inputs, targets = data
            outputs = model(inputs)  # Adjust as needed
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    average_loss = total_loss / len(test_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Test Loss: {average_loss}')

# モデルの保存
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)


TypeError: only integer scalar arrays can be converted to a scalar index

In [19]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming your DataFrame has a target variable named 'λmax'
target_column = df['λmax'].values.astype('float32')

# Split the data into training and testing sets
df_train, df_test, y_train, y_test = train_test_split(df, target_column, test_size=0.2, random_state=42)

# Convert SMILES strings to atom features for both training and testing sets
X_train = [smiles_to_atom_feature(smiles, res_type='tensor') for smiles in df_train['SMILES']]
X_test = [smiles_to_atom_feature(smiles, res_type='tensor') for smiles in df_test['SMILES']]

# Find the maximum number of atoms in the training and testing sets
max_atoms_train = max(x.size(0) for x in X_train)
max_atoms_test = max(x.size(0) for x in X_test)
max_atoms = max(max_atoms_train, max_atoms_test)

# Pad or trim the tensors to have the same size
X_train_padded = [torch.cat([x, torch.zeros(max_atoms - x.size(0), x.size(1))]) for x in X_train]
X_test_padded = [torch.cat([x, torch.zeros(max_atoms - x.size(0), x.size(1))]) for x in X_test]

In [40]:
# Convert SMILES strings to bond features for both training and testing sets
edge_index_train, edge_attr_train = zip(*[smiles_to_bond_feature(smiles) for smiles in df_train['SMILES']])
edge_index_test, edge_attr_test = zip(*[smiles_to_bond_feature(smiles) for smiles in df_test['SMILES']])

# Find the maximum number of bonds in the training and testing sets
max_bonds_train = max(attr.size(0) for attr in edge_attr_train)
max_bonds_test = max(attr.size(0) for attr in edge_attr_test)
max_bonds = max(max_bonds_train, max_bonds_test)

# Pad or trim the tensors to have the same size
edge_attr_train_padded = [torch.cat([attr, torch.zeros(max_bonds - attr.size(0), attr.size(1))]) for attr in edge_attr_train]
edge_attr_test_padded = [torch.cat([attr, torch.zeros(max_bonds - attr.size(0), attr.size(1))]) for attr in edge_attr_test]

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(torch.stack(X_train_padded), torch.stack(edge_attr_train_padded))
test_dataset = TensorDataset(torch.stack(X_test_padded), torch.stack(edge_attr_test_padded))

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Prepare edge_index and edge_attr using smiles_to_bond_feature function
edge_indices = []
edge_attrs = []

for smiles in tqdm(smiles_column):
    edge_index, edge_attr = smiles_to_bond_feature(smiles)
    edge_indices.append(edge_index)
    edge_attrs.append(edge_attr)

In [42]:
# モデルの構築
model = MolecularGCN2(61, 12, num_conv_layers=5, hidden_dim1=70, hidden_dim2=162, aggr='add')

# 損失関数の定義
criterion = nn.MSELoss()

# Convert edge indices and attributes to PyTorch tensors
edge_indices = [torch.tensor(idx, dtype=torch.long) for idx in edge_indices]
edge_attrs = [torch.tensor(attr, dtype=torch.float32) for attr in edge_attrs]

# 最適化アルゴリズムの選択
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_indices = list(range(len(molecules) // 2))  # adjust the split as needed
test_indices = list(range(len(molecules) // 2, len(molecules)))

# 学習のループ
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    for data in train_loader:
        inputs, edge_index, edge_attr, targets = data
        optimizer.zero_grad()
        outputs = model(inputs, edge_index, edge_attr, batch=None)  # Adjust batch as needed
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # エポックごとにモデルの評価
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for data in test_loader:
            inputs, edge_index, edge_attr, targets = data
            outputs = model(inputs, edge_index, edge_attr, batch=None)  # Adjust batch as needed
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    average_loss = total_loss / len(test_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Test Loss: {average_loss}')

# モデルの保存
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\Users\user\AppData\Local\Temp\ipykernel_25764\3365594984.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_indices = [torch.tensor(idx, dtype=torch.long) for idx in edge_indices]
C:\Users\user\AppData\Local\Temp\ipykernel_25764\3365594984.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attrs = [torch.tensor(attr, dtype=torch.float32) for attr in edge_attrs]
  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# モデルの構築
model = MolecularGCN2(61, 12, num_conv_layers=5, hidden_dim1=70, hidden_dim2=162, aggr='add')
model = model.to(device)  # Move the model to GPU

# 損失関数の定義
criterion = nn.MSELoss()

# 最適化アルゴリズムの選択
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習のループ
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    for data in train_loader:
        inputs, edge_index, edge_attr, targets = data
        inputs, edge_index, edge_attr, targets = inputs.to(device), edge_index.to(device), edge_attr.to(device), targets.to(device)  # Move the data to GPU
        optimizer.zero_grad()
        outputs = model(inputs, edge_index, edge_attr, batch=None)  # Adjust batch as needed
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # エポックごとにモデルの評価
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for data in test_loader:
            inputs, edge_index, edge_attr, targets = data
            inputs, edge_index, edge_attr, targets = inputs.to(device), edge_index.to(device), edge_attr.to(device), targets.to(device)  # Move the data to GPU
            outputs = model(inputs, edge_index, edge_attr, batch=None)  # Adjust batch as needed
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    average_loss = total_loss / len(test_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Test Loss: {average_loss}')

# モデルの保存
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)


  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 2)

In [38]:
# Convert SMILES strings to bond features for both training and testing sets
edge_index_train, edge_attr_train = zip(*[smiles_to_bond_feature(smiles) for smiles in df_train['SMILES']])
edge_index_test, edge_attr_test = zip(*[smiles_to_bond_feature(smiles) for smiles in df_test['SMILES']])

# Find the maximum number of bonds in the training and testing sets
max_bonds_train = max(attr.size(0) for attr in edge_attr_train)
max_bonds_test = max(attr.size(0) for attr in edge_attr_test)
max_bonds = max(max_bonds_train, max_bonds_test)

# Pad or trim the tensors to have the same size
edge_attr_train_padded = [torch.cat([attr, torch.zeros(max_bonds - attr.size(0), attr.size(1))]) for attr in edge_attr_train]
edge_attr_test_padded = [torch.cat([attr, torch.zeros(max_bonds - attr.size(0), attr.size(1))]) for attr in edge_attr_test]

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(torch.stack(X_train_padded), torch.stack(edge_index_train), torch.stack(edge_attr_train_padded), torch.from_numpy(y_train))
test_dataset = TensorDataset(torch.stack(X_test_padded), torch.stack(edge_index_test), torch.stack(edge_attr_test_padded), torch.from_numpy(y_test))

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 以下のコードは変更なし...


RuntimeError: stack expects each tensor to be equal size, but got [2, 42] at entry 0 and [2, 24] at entry 1

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# モデルの構築
model = MolecularGCN2(61, 12, num_conv_layers=5, hidden_dim1=70, hidden_dim2=162, aggr='add')
model = model.to(device)  # Move the model to GPU

# 損失関数の定義
criterion = nn.MSELoss()

# 最適化アルゴリズムの選択
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習のループ
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    for data in train_loader:
        inputs, edge_index, edge_attr, targets = data
        inputs, edge_index, edge_attr, targets = inputs.to(device), edge_index.to(device), edge_attr.to(device), targets.to(device)  # Move the data to GPU
        optimizer.zero_grad()
        outputs = model(inputs, edge_index, edge_attr, batch=None)  # Adjust batch as needed
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # エポックごとにモデルの評価
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for data in test_loader:
            inputs, edge_index, edge_attr, targets = data
            inputs, edge_index, edge_attr, targets = inputs.to(device), edge_index.to(device), edge_attr.to(device), targets.to(device)  # Move the data to GPU
            outputs = model(inputs, edge_index, edge_attr, batch=None)  # Adjust batch as needed
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    average_loss = total_loss / len(test_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Test Loss: {average_loss}')

# モデルの保存
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)
